In [0]:
%run ./Invoice_medallion_architecture_code

In [0]:
import time

class TestSuite:

    def __init__(self):
        self.invoice_dir = '/FileStore/invoices'
    
    def cleaning_dir(self):
        print('Cleaning Directory...', end='')
        spark.sql('drop table if exists invoice_bronze_table')
        spark.sql('drop table if exists invoice_silver_table')
        dbutils.fs.rm(f'{self.invoice_dir}/data', recurse=True)
        dbutils.fs.rm(f'{self.invoice_dir}/checkpoint_location', recurse=True)
        dbutils.fs.mkdirs(f'{self.invoice_dir}/data/bronze')
        print('Done')
    
    def copying_files(self, file_name):
        dbutils.fs.cp(f'{self.invoice_dir}/raw/{file_name}', f'{self.invoice_dir}/data/bronze', recurse=True)
    
    def assert_tables(self, table_name, expected_count):
        val = spark.sql(f"select count(*) from {table_name}").collect()[0][0]
        assert val==expected_count, f'Failed, returned value is {val}'
        print('Passed.')
    
    def test_case(self):
        self.cleaning_dir()
        bronze_obj = InvoiceBronze()
        BSquery = bronze_obj.launcher()

        silver_obj = InvoiceSilver()
        SSquery = silver_obj.launcher()

        print('\nTesting invoices_1.json file:')
        self.copying_files(file_name = 'invoices_1.json')
        print('Sleeping for 10sec...', end='')
        time.sleep(10)
        print('Done.')
        print('Testing bronze layer: ', end='')
        self.assert_tables('invoice_silver_table', 501)
        print('Testing silver layer: ', end='')
        self.assert_tables('invoice_gold_table', 1253)

        print('\nTesting invoices_2.json file:')
        self.copying_files(file_name = 'invoices_2.json')
        print('Sleeping for 10sec...', end='')
        time.sleep(10)
        print('Done.')
        print('Testing bronze layer: ', end='')
        self.assert_tables('invoice_silver_table', 1001)
        print('Testing silver layer: ', end='')
        self.assert_tables('invoice_gold_table', 2510)

        print('\nTesting invoices_3.json file:')
        self.copying_files(file_name = 'invoices_3.json')
        print('Sleeping for 10sec...', end='')
        time.sleep(10)
        print('Done.')
        print('Testing bronze layer: ', end='')
        self.assert_tables('invoice_silver_table', 1591)
        print('Testing silver layer: ', end='')
        self.assert_tables('invoice_gold_table', 3994)

        print('\nTesting invoices_4.json file:')
        self.copying_files(file_name = 'invoices_4.json')
        print('Sleeping for 10sec...', end='')
        time.sleep(10)
        print('Done.')
        print('Testing bronze layer: ', end='')
        self.assert_tables('invoice_silver_table', 1601)
        print('Testing silver layer: ', end='')
        self.assert_tables('invoice_gold_table', 4016)
        
        ls=['invoices_1.json', 'invoices_2.json', 'invoices_3.json']
        print('\nTesting archive files:', end='')
        for i in dbutils.fs.ls(f'{self.invoice_dir}/data/archive/{self.invoice_dir}/data/bronze'):
            assert i[1] in ls, f'Archive the {file} is failed.'
        print('Done.')

        BSquery.stop()
        SSquery.stop()

In [0]:
obj = TestSuite()
obj.test_case()

Cleaning Directory...Done
Bronze ingestion stream started...Done.
Silver Transformation stream started...Done.

Testing invoices_1.json file:
Sleeping for 10sec...Done.
Testing bronze layer: Passed.
Testing silver layer: Passed.

Testing invoices_2.json file:
Sleeping for 10sec...Done.
Testing bronze layer: Passed.
Testing silver layer: Passed.

Testing invoices_3.json file:
Sleeping for 10sec...Done.
Testing bronze layer: Passed.
Testing silver layer: Passed.

Testing invoices_4.json file:
Sleeping for 10sec...Done.
Testing bronze layer: Passed.
Testing silver layer: Passed.

Testing archive files:Done.
